[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zach-blumenfeld/graph-ml-examples/blob/main/graph-transformer-gps-pyg.ipynb)

Based off of [this code example](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/graph_gps.py)

In [1]:
%pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.2 MB/s eta 0:00:00


In [2]:
import argparse
import os.path as osp
from typing import Any, Dict, Optional

import torch
from torch.nn import (
    BatchNorm1d,
    Embedding,
    Linear,
    ModuleList,
    ReLU,
    Sequential,
)
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch_geometric.transforms as T
from torch_geometric.datasets import ZINC
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, GPSConv, global_add_pool
from torch_geometric.nn.attention import PerformerAttention

In [3]:
transform = T.AddRandomWalkPE(walk_length=20, attr_name='pe')
train_dataset = ZINC('.', subset=True, split='train', pre_transform=transform)
val_dataset = ZINC('.', subset=True, split='val', pre_transform=transform)
test_dataset = ZINC('.', subset=True, split='test', pre_transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

Extracting ./molecules.zip
Processing...
Processing test dataset: 100%|██████████| 1000/1000 [00:00<00:00, 1442.06it/s]
Done!


In [5]:
class RedrawProjection:
    def __init__(self, model: torch.nn.Module,
                 redraw_interval: Optional[int] = None):
        self.model = model
        self.redraw_interval = redraw_interval
        self.num_last_redraw = 0

    def redraw_projections(self):
        if not self.model.training or self.redraw_interval is None:
            return
        if self.num_last_redraw >= self.redraw_interval:
            fast_attentions = [
                module for module in self.model.modules()
                if isinstance(module, PerformerAttention)
            ]
            for fast_attention in fast_attentions:
                fast_attention.redraw_projection_matrix()
            self.num_last_redraw = 0
            return
        self.num_last_redraw += 1

class GPS(torch.nn.Module):
    def __init__(self, channels: int, pe_dim: int, num_layers: int,
                 attn_type: str, attn_kwargs: Dict[str, Any]):
        super().__init__()

        self.node_emb = Embedding(28, channels - pe_dim)
        self.pe_lin = Linear(20, pe_dim)
        self.pe_norm = BatchNorm1d(20)
        self.edge_emb = Embedding(4, channels)

        self.convs = ModuleList()
        for _ in range(num_layers):
            nn = Sequential(
                Linear(channels, channels),
                ReLU(),
                Linear(channels, channels),
            )
            conv = GPSConv(channels, GINEConv(nn), heads=4,
                           attn_type=attn_type, attn_kwargs=attn_kwargs)
            self.convs.append(conv)

        self.mlp = Sequential(
            Linear(channels, channels // 2),
            ReLU(),
            Linear(channels // 2, channels // 4),
            ReLU(),
            Linear(channels // 4, 1),
        )
        self.redraw_projection = RedrawProjection(
            self.convs,
            redraw_interval=1000 if attn_type == 'performer' else None)

    def forward(self, x, pe, edge_index, edge_attr, batch):
        x_pe = self.pe_norm(pe)
        x = torch.cat((self.node_emb(x.squeeze(-1)), self.pe_lin(x_pe)), 1)
        edge_attr = self.edge_emb(edge_attr)

        for conv in self.convs:
            x = conv(x, edge_index, batch, edge_attr=edge_attr)
        x = global_add_pool(x, batch)
        return self.mlp(x)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
attn_kwargs = {'dropout': 0.5}
model = GPS(channels=64, pe_dim=8, num_layers=10, attn_type='multihead',
            attn_kwargs=attn_kwargs).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        model.redraw_projection.redraw_projections()
        out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)
        loss = (out.squeeze() - data.y).abs().mean()
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_error = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)
        total_error += (out.squeeze() - data.y).abs().sum().item()
    return total_error / len(loader.dataset)


for epoch in range(1, 101):
    loss = train()
    val_mae = test(val_loader)
    test_mae = test(test_loader)
    scheduler.step(val_mae)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}')


Epoch: 01, Loss: 0.6875, Val: 0.4911, Test: 0.5251
Epoch: 02, Loss: 0.5468, Val: 0.4072, Test: 0.4013
Epoch: 03, Loss: 0.4700, Val: 0.4564, Test: 0.4476
Epoch: 04, Loss: 0.4663, Val: 0.5213, Test: 0.5009
Epoch: 05, Loss: 0.4458, Val: 0.3821, Test: 0.3792
Epoch: 06, Loss: 0.4052, Val: 0.3914, Test: 0.4183
Epoch: 07, Loss: 0.4277, Val: 0.3724, Test: 0.3445
Epoch: 08, Loss: 0.3761, Val: 0.4707, Test: 0.4439
Epoch: 09, Loss: 0.3823, Val: 0.3448, Test: 0.3016
Epoch: 10, Loss: 0.3654, Val: 0.3727, Test: 0.3486
Epoch: 11, Loss: 0.3628, Val: 0.5614, Test: 0.5008
Epoch: 12, Loss: 0.3436, Val: 0.3257, Test: 0.2987
Epoch: 13, Loss: 0.3450, Val: 0.3102, Test: 0.3114
Epoch: 14, Loss: 0.3409, Val: 0.3398, Test: 0.3023
Epoch: 15, Loss: 0.3447, Val: 0.3138, Test: 0.2813
Epoch: 16, Loss: 0.3452, Val: 0.4588, Test: 0.4463
Epoch: 17, Loss: 0.3265, Val: 0.3144, Test: 0.2898
Epoch: 18, Loss: 0.3396, Val: 0.4102, Test: 0.3722
Epoch: 19, Loss: 0.3226, Val: 0.4392, Test: 0.4152
Epoch: 20, Loss: 0.3144, Val: 0